In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [26]:
import numpy as np
import pandas as pd
import re

from imblearn.over_sampling import RandomOverSampler

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import SGDClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier

from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import  accuracy_score, precision_score, recall_score, f1_score

In [2]:
cd /content/drive/MyDrive/UBION_final_project/data

/content/drive/MyDrive/UBION_final_project/data


In [7]:
target = pd.read_csv('final_target.csv')
target = target[target['변수선택법']!='Lasso']
target.sort_values(by=['Recall', 'Accuracy'], ascending=[False, False], inplace=True)

In [12]:
df.columns

Index(['회사명', '거래소코드', '차기부도여부', '회계년도', '총자본증가율', '유형자산증가율', '유동자산증가율',
       '영업이익증가율', '순이익증가율', '재고자산증가율',
       ...
       'PCR(Price cash-flow ratio)(최저)', 'PSR(Price sales ratio)(최고)',
       'PSR(Price sales ratio)(최저)', '기업가치(EV)(백만원)', 'EBITDA(백만원)',
       'EBITDA/매출액(%)', 'EBITDA/금융비용(배)', 'EBITDA/평균발행주식수(백만원)',
       'EV/EBITDA(배)', '산업대분류'],
      dtype='object', length=162)

In [10]:
idx = target.iloc[0,:]
df = pd.read_csv(idx[0]+'.csv')

In [15]:
variables = idx[7]
variables = re.sub("\[|\]|\'|\'|", "",variables)
selected_variables_list = list(variables.split(', '))
selected_variables_list.append('차기부도여부')
selected_variables_list

['총자본사업이익률',
 '자기자본구성비율',
 '종업원수',
 '단기차입금 대 총차입금비율',
 'CASH FLOW 대 부채비율',
 'EBITDA(백만원)',
 '재고자산 대 유동자산비율',
 '인건비(백만원)',
 '자기자본증가율',
 '타인자본구성비율',
 '총자본순이익률',
 '기업순이익률',
 '차기부도여부']

In [20]:
df = df[selected_variables_list]

In [21]:
df['차기부도여부'].value_counts()

0    12087
1      198
Name: 차기부도여부, dtype: int64

In [24]:
data = df.drop(['차기부도여부'], axis= 1)
target = df['차기부도여부']

X_train, X_test, y_train, y_test = train_test_split(data, target, test_size=0.178, shuffle=True, stratify=target, random_state=34)

In [27]:
ros = RandomOverSampler()

X_oversampled, y_oversampled = ros.fit_resample(X_train, y_train)

In [28]:
lr = LogisticRegression(random_state=0)
lr.fit(X_oversampled, y_oversampled)

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


LogisticRegression(random_state=0)

In [29]:
prediction = lr.predict(X_test)
prediction_prob =lr.predict_proba(X_test)

In [68]:
y_test = pd.Series(y_test.values)

In [73]:
y_test = y_test.to_frame()

In [76]:
result_df = pd.DataFrame(prediction_prob, columns=['정상_prob','부도_prob'])
result_df['prediction'] = prediction
result_df['Actual'] = y_test

In [77]:
result_df

,정상_prob,부도_prob,prediction,Actual
0,0.926238,0.073762,0,0
1,0.155306,0.844694,1,0
2,0.995701,0.004299,0,0
3,0.966026,0.033974,0,0
4,0.732273,0.267727,0,0
...,...,...,...,...
2182,0.434483,0.565517,1,0
2183,0.618095,0.381905,0,0
2184,0.987676,0.012324,0,0
2185,0.953348,0.046652,0,0


In [56]:
len(result_df)

2187

In [54]:
y_test.value_counts()

0    2152
1      35
Name: 차기부도여부, dtype: int64

In [78]:
result_df['Actual'].value_counts()

0    2152
1      35
Name: Actual, dtype: int64